In [6]:
import os
import random
import zipfile
from matplotlib import pyplot as plt
from PIL import Image

In [2]:
slices = [50, 50, 50]

In [5]:
for subdir, dirs, files in os.walk('/Volumes/Extreme SSD/ADNI_SLICED/'):

    images = files

    num_of_images = len(files)

    image1 = files[random.randint(0, num_of_images)]

    #image2 = files[random.randint(0, num_of_images)]

    #image3 = files[random.randint(0, num_of_images)]

    image_archive1 = zipfile.ZipFile('/Volumes/Extreme SSD/ADNI_SLICED/'+image1)

    image_archive1.extract('saggital/'+slices[0]+'.png', 'temp_folder_saggital')

    image_archive1.extract('axial/'+slices[1]+'.png', 'temp_folder_axial')

    image_archive1.extract('coronal/'+slices[2]+'.png', 'temp_folder_coronal')

    #image_archive2 = zipfile.ZipFile('/Volumes/Extreme SSD/ADNI_SLICED/'+image2)

    ##image_archive3 = zipfile.ZipFile('/Volumes/Extreme SSD/ADNI_SLICED/'+image3)

    image1 = Image.open(r"temp_folder_saggital")
 
    image2 = Image.open(r"temp_folder_axial")

    image3 = Image.open(r"temp_folder_coronal")

    fig = plt.figure(figsize=(10, 10))
    plt.title('Image')

    for i in range(1, 2):
        fig.add_subplot(3, 3, i)
        plt.imshow('image',i)

    plt.show()
